In [1]:
# Rotten Tomatoes Parser. Will obtain RT score and Audience Score, Director, Writers, and Top Billed Actors/Actresses
from pymongo import MongoClient
import pandas as pd
import numpy as np
import random

# Requests sends and receives HTTP requests
import requests
from bs4 import BeautifulSoup

#used if i need wildcard in bs
import re

import time
import pprint

In [2]:
client = MongoClient('localhost', 27017)
db = client['scraper']
whole_page_ratings = db['whole_page_rt']
rt_ratings = db['rt_ratings']

In [3]:
# Pull in cleaned movie list
budgets_df = pd.read_csv('data/budgets_pd.csv')
movies = budgets_df['title'].tolist()

movies_copy = movies.copy()

cleaned = []
for string in movies_copy:
    clean = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),string)
    cleaned.append(clean)
print(cleaned)

['Avengers: Endgame', 'Pirates of the Caribbean: On Stranger Tides', 'Avengers: Age of Ultron', 'Star Wars Ep. VII: The Force Awakens', 'Avengers: Infinity War', 'Pirates of the Caribbean: At World’s End', 'Justice League', 'Spectre', 'Star Wars: The Rise of Skywalker', 'Solo: A Star Wars Story', 'John Carter', 'Batman v Superman: Dawn of Justice', 'The Lion King', 'Tangled', 'Spider-Man 3', 'Captain America: Civil War', 'Harry Potter and the Half-Blood Prince', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: The Battle of the Five Armies', 'The Fate of the Furious', 'No Time to Die', 'Avatar', 'Superman Returns', 'The Dark Knight Rises', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'Quantum of Solace', 'The Avengers', 'Pirates of the Caribbean: Dead Man’s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Lone Ranger', 'Tenet', 'The Amazing Spider-Man', 'Battleship', 'Transformers: The Last Knight', 'Jurassic World', 'Men in Black 3', 'Transformers: Re

In [167]:
formatted = []
for movie in cleaned:
    _ = movie.replace(': ', '_')
    add = _.replace(' ', '_')
    add = add.replace('-', '_')
    add = add.replace('.', '')
    add = add.replace(',', '')
    
    formatted.append(add)
    
print(formatted)

['Avengers_Endgame', 'Pirates_of_the_Caribbean_On_Stranger_Tides', 'Avengers_Age_of_Ultron', 'Star_Wars_Ep_VII_The_Force_Awakens', 'Avengers_Infinity_War', 'Pirates_of_the_Caribbean_At_World’s_End', 'Justice_League', 'Spectre', 'Star_Wars_The_Rise_of_Skywalker', 'Solo_A_Star_Wars_Story', 'John_Carter', 'Batman_v_Superman_Dawn_of_Justice', 'The_Lion_King', 'Tangled', 'Spider_Man_3', 'Captain_America_Civil_War', 'Harry_Potter_and_the_Half_Blood_Prince', 'The_Hobbit_The_Desolation_of_Smaug', 'The_Hobbit_The_Battle_of_the_Five_Armies', 'The_Fate_of_the_Furious', 'No_Time_to_Die', 'Avatar', 'Superman_Returns', 'The_Dark_Knight_Rises', 'Pirates_of_the_Caribbean_Dead_Men_Tell_No_Tales', 'Quantum_of_Solace', 'The_Avengers', 'Pirates_of_the_Caribbean_Dead_Man’s_Chest', 'Man_of_Steel', 'The_Chronicles_of_Narnia_Prince_Caspian', 'The_Lone_Ranger', 'Tenet', 'The_Amazing_Spider_Man', 'Battleship', 'Transformers_The_Last_Knight', 'Jurassic_World', 'Men_in_Black_3', 'Transformers_Revenge_of_the_Falle

In [13]:
# Ex. result URL https://www.rottentomatoes.com/m/avengers_endgame
main_url= 'https://www.rottentomatoes.com/m/'
links = []
for movie in formatted:
    link = f'{main_url}{movie}'
    links.append(link)
print(links[0])

https://www.rottentomatoes.com/m/Avengers_Endgame


In [ ]:
# If request fails make sure to try to grab these movies again
try_again = []

In [106]:
r = requests.get(links[1])
r.status_code
if r.status_code != 200:
    try_again.append(movie)

In [107]:
pprint.pprint(r.text)

('<!DOCTYPE html>\n'
 '<html lang="en"\n'
 '      dir="ltr"\n'
 '      xmlns:fb="http://www.facebook.com/2008/fbml"\n'
 '      xmlns:og="http://opengraphprotocol.org/schema/">\n'
 '\n'
 '    <head prefix="og: http://ogp.me/ns# flixstertomatoes: '
 'http://ogp.me/ns/apps/flixstertomatoes#">\n'
 '\n'
 '        <!-- salt=lay-def-02-juRm -->\n'
 '        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" '
 '/>\n'
 '        <meta http-equiv="x-ua-compatible" content="ie=edge">\n'
 '        <meta name="viewport" content="width=device-width, '
 'initial-scale=1">\n'
 '\n'
 '        <title>Pirates of the Caribbean: On Stranger Tides (2011) - Rotten '
 'Tomatoes</title>\n'
 '        <meta name="description" content="Johnny Depp returns to his iconic '
 'role of Captain Jack Sparrow in an action-packed adventure. Crossing paths '
 "with the enigmatic Angelica (Penélope Cruz), he's not sure if it's love-or "
 "if she's a ruthless con artist who's using him to find the fabled Foun

 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/atlanta/s02/#desktopEpisodeList"> Atlanta: Robbin&#39; Season '
 '</a></li>\n'
 '                                        </div>\n'
 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/fargo/s03/#desktopEpisodeList"> Fargo: Season 3 </a></li>\n'
 '                                        </div>\n'
 '                                        \n'
 '                                        <div '
 'class="header-desktop-episode">\n'
 '                                            <li><a '
 'href="/tv/the_flash/s06/#desktopEpisodeList"> The Flash: Season 6 </a></li>\n'
 '                                        </div>\n'
 '                                    

 '        View All<span class="glyphicon" />\n'
 '    </a>\n'
 '</div>\n'
 '\n'
 '            </section>\n'
 '            <section class="tab-pane fade " id="on-tv">\n'
 '                \n'
 '    <h3>New TV Tonight</h3>\n'
 '    <table class="movie_list tv_list">\n'
 '        \n'
 '            \n'
 '            \n'
 '            <tr class="tv_show_tr tvList1">\n'
 '    <td class="left_col">\n'
 '        <a href="/tv/stargirl/s01">\n'
 '            \n'
 '                <span class="icon tiny certified_fresh"></span>\n'
 '                <span class="tMeterScore">91%</span>\n'
 '            \n'
 '        </a>\n'
 '    </td>\n'
 '    <td class="middle_col">\n'
 '        <a href="/tv/stargirl/s01">\n'
 '            \n'
 '                Stargirl\n'
 '            \n'
 '        </a>\n'
 '    </td>\n'
 '</tr>\n'
 '\n'
 '         \n'
 '            \n'
 '            \n'
 '            <tr class="tv_show_tr tvList1">\n'
 '    <td class="left_col">\n'
 '        <a href="/tv/greenleaf/s05">\n'
 '

 'js-user-rating-error-close-button"></button>\n'
 '                    <h3 class="user-rating-error__header '
 'js-user-rating-error-header"></h3>\n'
 '                    <div class="user-rating-error__body '
 'js-user-rating-error-text"></div>\n'
 '                </section>\n'
 '            </div>\n'
 '\n'
 '            \n'
 '            \n'
 '    \n'
 '    \n'
 '    \n'
 '        \n'
 '    \n'
 '    <aside id="interscroller_ad" class="interscroller_ad visible-xs center '
 'mobile-interscroller " style="width:300px"></aside>\n'
 '    <script>\n'
 '        var mps = mps || {};\n'
 '        mps._queue = mps._queue || {};\n'
 '        mps._queue.gptloaded = mps._queue.gptloaded || [];\n'
 '        mps._queue.adload = mps._queue.adload || [];\n'
 '\n'
 '        mps._queue.gptloaded.push(function() {\n'
 "            if (mps.getResponsiveSet() =='0') { //MOBILE\n"
 "                mps.insertAd('#interscroller_ad','interscroller');\n"
 '            }\n'
 '        });\n'
 '        mps._q

 '        </span>\n'
 '    </div>\n'
 '</div>\n'
 '\n'
 '        \n'
 '            \n'
 '            \n'
 '            <div class="cast-item media inlineBlock  castRemaining">\n'
 '    <a href="/celebrity/brea_berrett" class="pull-left">\n'
 '        \n'
 '            <img '
 'data-src="/assets/pizza-pie/images/poster_default_thumbnail.2ec144e61b4.jpg" '
 'class="js-lazyLoad actorThumb medium media-object" />\n'
 '        \n'
 '    </a>\n'
 '    <div class="media-body">\n'
 '        <a href="/celebrity/brea_berrett" class="unstyled articleLink">\n'
 '            <span title="Brea Berrett">\n'
 '                Brea Berrett\n'
 '            </span>\n'
 '        </a>\n'
 '        <span class="characters subtle smaller" title="Mermaid">\n'
 '            \n'
 '            <br/>as\n'
 '                \n'
 '                    Mermaid\n'
 '                \n'
 '            \n'
 '        </span>\n'
 '    </div>\n'
 '</div>\n'
 '\n'
 '        \n'
 '            \n'
 '            \n'
 '        

 '            \n'
 '        </div>\n'
 '    </div>\n'
 '</aside>\n'
 '\n'
 '\n'
 '        \n'
 '        \n'
 '\n'
 '    <script '
 'src="//player.theplatform.com/pdk/dCK2IC/tpPdkController.js?pdk=5.7.14"></script>\n'
 '    </section>\n'
 '\n'
 '            </div>\n'
 '\n'
 '            \n'
 '    \n'
 '    \n'
 '    <div class="sleaderboard_wrapper hidden-xs mobile-hidden">\n'
 '    <div id="leaderboard_bottom_ad" '
 'style="margin-left:auto;margin-right:auto;display:inline-block"> </div>\n'
 '    <script>\n'
 '        var mps = mps||{}; mps._queue = mps._queue||{}; mps._queue.gptloaded '
 '= mps._queue.gptloaded||[];\n'
 '        mps._queue.gptloaded.push(function () {\n'
 "            if (mps.getResponsiveSet() != '0') { //DESKTOP or TABLET\n"
 "                mps.insertAd('#leaderboard_bottom_ad','bottombanner');\n"
 '            }\n'
 '        });\n'
 '    </script>\n'
 '    </div>\n'
 '\n'
 '\n'
 '\n'
 '            \n'
 '            \n'
 '                \n'
 '    \n'
 '        \n

In [108]:
soup = BeautifulSoup(r.text, 'html.parser')

In [109]:
for rating in soup.find_all('h2', class_="mop-ratings-wrap__score"):
    print(rating.text)





                    33%
                






                    54%
                




In [110]:
# RT rating
rt_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[0].text

# Audience rating
aud_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[1].text

In [111]:
# RT Cleaning
rt_rating = rt_rating.replace("  ", '')
rt_rating = rt_rating.replace("\n", '')
print(rt_rating)

33%


In [112]:
# Audience Cleaning
aud_rating = aud_rating.replace("  ", '')
aud_rating = aud_rating.replace("\n", '')
print(aud_rating)

54%


In [113]:
# Director NEEDS CLEANING
director = soup.find_all('li', class_="meta-row clearfix")[2].text

In [120]:
# Director Cleaned
director = director.replace("  ", '')
director = director.replace("\n", '')
director = director.replace("Directed By:", '')
director = director.split(',')
print(director)

['Anthony Russo']


In [115]:
# Writers NEEDS CLEANING
writers = soup.find_all('li', class_="meta-row clearfix")[3].text

In [116]:
# Writers Cleaned

writers = writers.replace("  ", '')
writers = writers.replace("\n", '')
writers = writers.replace("Written By:", '')
writers = writers.split(',')
print(writers)

['Rob Marshall', ' Terry Rossio', ' Ted Elliott']


In [117]:
# Top 5 billed Actors/Actresses

test_ = soup.find_all('div', class_="castSection")[0].text
test = test_.split("  \n  ")
# print(test)
names = []
for name in test:
    name = name.replace('\nas\n ', '')
    name = name.replace('\n', '')
    name = name.replace('  ', '')
    names.append(name)


act = names[0:13:3]
print(act)

['Johnny Depp ', 'Penelope Cruz ', 'Ian McShane ', 'Geoffrey Rush ', 'Kevin McNally ']


In [169]:
# Set up loop for requests
# Rotten Tomatoes Parser. Will obtain RT score and Audience Score, Director, Writers, and Top Billed Actors/Actresses
from pymongo import MongoClient
import pandas as pd
import numpy as np
import random

# Requests sends and receives HTTP requests
import requests
from bs4 import BeautifulSoup

#used if i need wildcard in bs
import re

import time
import pprint

In [170]:
client = MongoClient('localhost', 27017)
db = client['scraper']
whole_page_ratings = db['whole_page_rt']
rt_ratings = db['rt_ratings']

In [171]:
# Pull in cleaned movie list
budgets_df = pd.read_csv('data/budgets_pd.csv')
movies = budgets_df['title'].tolist()

movies_copy = movies.copy()

cleaned = []
for string in movies_copy:
    clean = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),string)
    cleaned.append(clean)
print(cleaned)

try_again = []

['Avengers: Endgame', 'Pirates of the Caribbean: On Stranger Tides', 'Avengers: Age of Ultron', 'Star Wars Ep. VII: The Force Awakens', 'Avengers: Infinity War', 'Pirates of the Caribbean: At World’s End', 'Justice League', 'Spectre', 'Star Wars: The Rise of Skywalker', 'Solo: A Star Wars Story', 'John Carter', 'Batman v Superman: Dawn of Justice', 'The Lion King', 'Tangled', 'Spider-Man 3', 'Captain America: Civil War', 'Harry Potter and the Half-Blood Prince', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: The Battle of the Five Armies', 'The Fate of the Furious', 'No Time to Die', 'Avatar', 'Superman Returns', 'The Dark Knight Rises', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'Quantum of Solace', 'The Avengers', 'Pirates of the Caribbean: Dead Man’s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Lone Ranger', 'Tenet', 'The Amazing Spider-Man', 'Battleship', 'Transformers: The Last Knight', 'Jurassic World', 'Men in Black 3', 'Transformers: Re

In [220]:
formatted = []
for movie in cleaned:
    _ = movie.replace(': ', '_')
    add = _.replace(' ', '_')
    add = add.replace('-', '_')
    add = add.replace('.', '')
    add = add.replace(',', '')
    add = add.replace('&', 'and')
    formatted.append(add)
    
print(formatted)

['Star_Wars_Ep_VII_The_Force_Awakens', 'Pirates_of_the_Caribbean_At_World’s_End', 'Justice_League', 'Spectre', 'Spider_Man_3', 'No_Time_to_Die', 'The_Avengers', 'Pirates_of_the_Caribbean_Dead_Man’s_Chest', 'Man_of_Steel', 'The_Chronicles_of_Narnia_Prince_Caspian', 'Tenet', 'Men_in_Black_3', 'Robin_Hood', 'Black_Panther', 'Star_Wars_Ep_VIII_The_Last_Jedi', 'Spider_Man_2', 'Alice_in_Wonderland', 'Fast_and_Furious_Presents_Hobbs_and_Shaw', 'Tron_Legacy', 'The_Mummy', 'The_Great_Gatsby', 'Disney’s_A_Christmas_Carol', 'Pacific_Rim', 'Brave', 'The_Chronicles_of_Narnia_The_Lion_the_Witch_a…', 'Mission_Impossible—Fallout', 'The_Jungle_Book', 'Inside_Out', 'Suicide_Squad', 'Coco', 'The_Mummy_Tomb_of_the_Dragon_Emperor', '47_Ronin', 'The_Polar_Express', 'Tomorrowland', 'Alice_Through_the_Looking_Glass', 'Doctor_Strange', 'Beauty_and_the_Beast', 'The_Hunger_Games_Mockingjay___Part_2', 'The_Curious_Case_of_Benjamin_Button', "The_Sorcerer's_Apprentice", 'The_Chronicles_of_Narnia_The_Voyage_of_the_D

In [ ]:
# Requests
# Ex. result URL https://www.rottentomatoes.com/m/avengers_endgame
main_url= 'https://www.rottentomatoes.com/m/'
links = []
for movie in formatted:
    link = f'{main_url}{movie}'
    links.append(link)

# If pervious fails due to movie year added in url
links_year = []
for idx, movie in enumerate(formatted):
    link = f'{main_url}{movie}_{y[idx]}'
    links_year.append(link)

# Loop for all pages     
for idx, link in enumerate(links):   
    try:
        print(link)
        r = requests.get(link)
        r.status_code
    except:
        print(links_year[idx])
        r = requests.get(links_year[idx])
        r.status_code
    if r.status_code != 200:
        try_again.append(missing[idx])
        slp = random.randint(5,15)
        time.sleep(slp)
        continue
    try:    
        soup = BeautifulSoup(r.text, 'html.parser')

        # Title
        title = missing[idx]

        # RT rating
        rt_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[0].text

        # RT Cleaning
        rt_rating = rt_rating.replace("  ", '')
        rt_rating = rt_rating.replace("\n", '')

        # Audience rating
        aud_rating = soup.find_all('h2', class_="mop-ratings-wrap__score")[1].text

        # Audience Cleaning
        aud_rating = aud_rating.replace("  ", '')
        aud_rating = aud_rating.replace("\n", '')

        # Director 
        director = soup.find_all('li', class_="meta-row clearfix")[2].text

        # Director Cleaning
        director = director.replace("  ", '')
        director = director.replace("\n", '')
        director = director.replace("Directed By:", '')
        director = director.split(',')

        # Writers
        writers = soup.find_all('li', class_="meta-row clearfix")[3].text

        # Writers Cleaning
        writers = writers.replace("  ", '')
        writers = writers.replace("\n", '')
        writers = writers.replace("Written By:", '')
        writers = writers.split(',')

        # Top 5 billed Actors/Actresses
        test_ = soup.find_all('div', class_="castSection")[0].text
        test = test_.split("  \n  ")
        names = []
        for name in test:
            name = name.replace('\nas\n ', '')
            name = name.replace('\n', '')
            name = name.replace('  ', '')
            names.append(name)
        act = names[0:13:3]

        # Add to Mongo db
        rt_ratings.insert_one({'title': title, 'rotten_tomato_rating':rt_rating, 'audience_rating':aud_rating, 'director':director, 'writers':writers, 'top_5_billed':act})
    except:
#         try_again.append(movies[idx])
        try_again.append(missing[idx])
    
    
    # Mongo print for sanity
    print(db.rt_ratings.find().count())
    
    # Add time before next request 
    slp = random.randint(5,15)
    time.sleep(slp)

https://www.rottentomatoes.com/m/Star_Wars_Ep_VII_The_Force_Awakens
https://www.rottentomatoes.com/m/Pirates_of_the_Caribbean_At_World’s_End
https://www.rottentomatoes.com/m/Justice_League
https://www.rottentomatoes.com/m/Spectre


/Users/Michael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


1372
https://www.rottentomatoes.com/m/Spider_Man_3
https://www.rottentomatoes.com/m/No_Time_to_Die
https://www.rottentomatoes.com/m/The_Avengers
https://www.rottentomatoes.com/m/Pirates_of_the_Caribbean_Dead_Man’s_Chest
https://www.rottentomatoes.com/m/Man_of_Steel
https://www.rottentomatoes.com/m/The_Chronicles_of_Narnia_Prince_Caspian
https://www.rottentomatoes.com/m/Tenet
1372
https://www.rottentomatoes.com/m/Men_in_Black_3
https://www.rottentomatoes.com/m/Robin_Hood
1372
https://www.rottentomatoes.com/m/Black_Panther
https://www.rottentomatoes.com/m/Star_Wars_Ep_VIII_The_Last_Jedi
https://www.rottentomatoes.com/m/Spider_Man_2
https://www.rottentomatoes.com/m/Alice_in_Wonderland
https://www.rottentomatoes.com/m/Fast_and_Furious_Presents_Hobbs_and_Shaw
1373
https://www.rottentomatoes.com/m/Tron_Legacy
https://www.rottentomatoes.com/m/The_Mummy
https://www.rottentomatoes.com/m/The_Great_Gatsby
1373
https://www.rottentomatoes.com/m/Disney’s_A_Christmas_Carol
https://www.rottentomatoes.

https://www.rottentomatoes.com/m/Spider_Man_Into_The_Spider_Verse_3D
https://www.rottentomatoes.com/m/Sonic_The_Hedgehog
https://www.rottentomatoes.com/m/Once_Upon_a_Time…in_Hollywood
https://www.rottentomatoes.com/m/Charlie's_Angels
https://www.rottentomatoes.com/m/Contact
1378
https://www.rottentomatoes.com/m/You_Don't_Mess_With_the_Zohan
https://www.rottentomatoes.com/m/Artificial_Intelligence_AI
https://www.rottentomatoes.com/m/The_Haunted_Mansion
https://www.rottentomatoes.com/m/The_Interpreter
1378
https://www.rottentomatoes.com/m/The_Saint
https://www.rottentomatoes.com/m/Tomb_Raider
https://www.rottentomatoes.com/m/King_Arthur
https://www.rottentomatoes.com/m/The_Devil's_Own
https://www.rottentomatoes.com/m/K_19_The_Widowmaker
https://www.rottentomatoes.com/m/The_Three_Musketeers
1378
https://www.rottentomatoes.com/m/The_Promise
https://www.rottentomatoes.com/m/Army_of_the_Dead
1378
https://www.rottentomatoes.com/m/Twister
https://www.rottentomatoes.com/m/The_Predator
1378
http

1386
https://www.rottentomatoes.com/m/The_Tuxedo
https://www.rottentomatoes.com/m/Vice
1386
https://www.rottentomatoes.com/m/Gangster_Squad
https://www.rottentomatoes.com/m/Duplicity
1386
https://www.rottentomatoes.com/m/State_of_Play
https://www.rottentomatoes.com/m/The_Sentinel
https://www.rottentomatoes.com/m/The_Relic
https://www.rottentomatoes.com/m/The_Soloist
https://www.rottentomatoes.com/m/The_Legend_of_Bagger_Vance
https://www.rottentomatoes.com/m/Garfield_A_Tail_of_Two_Kitties
https://www.rottentomatoes.com/m/The_Avengers
https://www.rottentomatoes.com/m/The_Black_Dahlia
https://www.rottentomatoes.com/m/The_Last_Castle
https://www.rottentomatoes.com/m/Supernova
1386
https://www.rottentomatoes.com/m/Winter’s_Tale
https://www.rottentomatoes.com/m/The_Great_Raid
https://www.rottentomatoes.com/m/Xi_You_Ji_zhi_Sun_Wu_Kong_San_Da_Bai_Gu_Jing
https://www.rottentomatoes.com/m/Monster_Hunter
1386
https://www.rottentomatoes.com/m/Goosebumps
https://www.rottentomatoes.com/m/Sabrina
138

In [136]:
#Debug area
print(soup.find_all('h2', class_="mop-ratings-wrap__score"))

[]


In [132]:
print(soup)

<!DOCTYPE html>

<html dir="ltr" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<!-- salt=lay-def-02-juRm -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Rotten Tomatoes: Movies - Rotten Tomatoes</title>
<meta content="Rotten Tomatoes, home of the Tomatometer, is the most trusted measurement of quality for Movies &amp; TV. The definitive site for Reviews, Trailers, Showtimes, and Tickets" name="description"/>
<link href="https://www.rottentomatoes.com/m/Star_Wars_Ep._VII_The_Force_Awakens" rel="canonical"/>
<link href="https://www.rottentomatoes.com/assets/pizza-pie/images/favicon.ico" rel="shortcut icon" sizes="76x76" type="image/x-icon"/>
<meta content="326803741017" property="fb

In [214]:
rt_pd = pd.DataFrame(list(rt_ratings.find()))

In [216]:
rt_pd.to_csv('data/rt_pd.csv', index=False)

In [177]:
print(try_again)
print(len(try_again))

['Star Wars Ep. VII: The Force Awakens', 'Pirates of the Caribbean: At Worldâ\x80\x99s End', 'Justice League', 'Spectre', 'Spider-Man 3', 'No Time to Die', 'The Avengers', 'Pirates of the Caribbean: Dead Manâ\x80\x99s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'Tenet', 'Men in Black 3', 'Robin Hood', 'Black Panther', 'Star Wars Ep. VIII: The Last Jedi', 'Spider-Man 2', 'Alice in Wonderland', 'Fast & Furious Presents: Hobbs & Shaw', 'Tron: Legacy', 'The Mummy', 'The Great Gatsby', 'Disneyâ\x80\x99s A Christmas Carol', 'Pacific Rim', 'Brave', 'The Chronicles of Narnia: The Lion, the Witch a…', 'Mission: Impossibleâ\x80\x94Fallout', 'The Jungle Book', 'Inside Out', 'Suicide Squad', 'Coco', 'The Mummy: Tomb of the Dragon Emperor', '47 Ronin', 'The Polar Express', 'Tomorrowland', 'Alice Through the Looking Glass', 'Doctor Strange', 'Beauty and the Beast', 'The Hunger Games: Mockingjay - Part 2', 'The Curious Case of Benjamin Button', "The Sorcerer's Apprentice", 'Th

In [217]:
rt_pd = pd.DataFrame(list(rt_ratings.find()))

In [147]:
rt_pd.head()

,_id,title,rotten_tomato_rating,audience_rating,director,writers,top_5_billed
0,5f17b668eec2a42d5d9cccf4,Avengers: Endgame,94%,90%,[Anthony Russo],"[Christopher Markus, Stephen McFeely]","[Robert Downey Jr. , Mark Ruffalo , Scarlett J..."
1,5f17b677eec2a42d5d9cccf5,Pirates of the Caribbean: On Stranger Tides,33%,54%,"[Dariusz Wolski, Rob Marshall]","[Rob Marshall, Terry Rossio, Ted Elliott]","[Johnny Depp , Penelope Cruz , Ian McShane , G..."
2,5f17b67eeec2a42d5d9cccf6,Avengers: Age of Ultron,76%,83%,[Joss Whedon],"[Stan Lee, Jack Kirby, Joss Whedon]","[Robert Downey Jr. , Chris Evans , Mark Ruffal..."
3,5f17b687eec2a42d5d9cccf7,Avengers: Infinity War,85%,91%,[Anthony Russo],"[Christopher Markus, Stephen McFeely]","[Robert Downey Jr. , Chris Hemsworth , Mark Ru..."
4,5f17b6a0eec2a42d5d9cccf8,Star Wars: The Rise of Skywalker,51%,86%,[J.J. Abrams],"[J.J. Abrams, Chris Terrio]","[Carrie Fisher , Mark Hamill , Adam Driver , D..."


In [218]:
rt_pd.tail()

,_id,title,rotten_tomato_rating,audience_rating,director,writers,top_5_billed
1367,5f18b786eec2a42d5d9cd3b5,Exodus: Gods and Kings,30%,35%,[Ridley Scott],"[Steven Zaillian, Bill Collage, Adam Cooper]","[Christian Bale , Ben Kingsley , Joel Edgerton..."
1368,5f18b80ceec2a42d5d9cd3b6,Australia,55%,65%,[Baz Luhrmann],"[Stuart Beattie, Baz Luhrmann, Ronald Harwoo...","[Hugh Jackman , Nicole Kidman , Jack Thompson ..."
1369,5f18b91aeec2a42d5d9cd3b7,Star Wars Ep. I: The Phantom Menace,53%,59%,[George Lucas],[George Lucas],"[Liam Neeson , Ewan McGregor , Natalie Portman..."
1370,5f18b923eec2a42d5d9cd3b8,Star Wars Ep. III: Revenge of the Sith,80%,66%,[George Lucas],[George Lucas],"[Ewan McGregor , Natalie Portman , Hayden Chri..."
1371,5f18b92ceec2a42d5d9cd3b9,Star Wars Ep. II: Attack of the Clones,65%,56%,[George Lucas],"[Jonathan Hales, George Lucas]","[Hayden Christensen , Natalie Portman , Ewan M..."


In [226]:
missing = []
for idx in range(len(movies)):
    if movies[idx] not in titles:
        missing.append(movies[idx])
print(len(missing))
    

4659


In [233]:
missing_df = pd.DataFrame(missing, columns=['title'])
budgets_pd = pd.read_csv('data/budgets_pd.csv')

In [234]:
merged= pd.merge(missing_df, budgets_pd, on='title')

In [237]:
merged.tail()

,title,_id,date,budget,domestic_gross,worldwide_gross
4784,Red 11,5f17b5918fbba2e9aaf4411d,Unknown,"$7,000",$0,$0
4785,Following,5f17b5918fbba2e9aaf4411e,"Apr 2, 1999","$6,000","$48,482","$240,495"
4786,Return to the Land of Wonders,5f17b5918fbba2e9aaf4411f,"Jul 13, 2005","$5,000","$1,338","$1,338"
4787,A Plague So Pleasant,5f17b5918fbba2e9aaf44120,"Sep 29, 2015","$1,400",$0,$0
4788,My Date With Drew,5f17b5918fbba2e9aaf44121,"Aug 5, 2005","$1,100","$181,041","$181,041"


In [238]:
years = get_year(merged['date'])

In [240]:
merged['year'] = years

In [242]:
y = merged.year.tolist()

In [243]:
len(y)

4789

In [186]:
print(movies[365], formatted[365])

I, Robot I_Robot


In [153]:
movies[356]

'Speed 2: Cruise Control'

In [190]:
print(len(rt_pd.title.unique()))

1353


In [200]:
titles = rt_pd['title'].tolist()

['Avengers: Endgame', 'Pirates of the Caribbean: On Stranger Tides', 'Avengers: Age of Ultron', 'Avengers: Infinity War', 'Star Wars: The Rise of Skywalker', 'Solo: A Star Wars Story', 'John Carter', 'Batman v Superman: Dawn of Justice', 'The Lion King', 'Tangled', 'Captain America: Civil War', 'Harry Potter and the Half-Blood Prince', 'The Hobbit: The Desolation of Smaug', 'The Hobbit: The Battle of the Five Armies', 'The Fate of the Furious', 'Avatar', 'Superman Returns', 'The Dark Knight Rises', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'Quantum of Solace', 'The Lone Ranger', 'The Amazing Spider-Man', 'Battleship', 'Transformers: The Last Knight', 'Jurassic World', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'X-Men: The Last Stand', 'King Kong', 'The Golden Compass', 'Titanic', 'Incredibles 2', 'Rogue One: A Star Wars Story', 'Finding Dory', 'Toy Story 4', 'Toy Story 3', 'Iron Man 3', 'Guardians of the Galaxy Vol 2', 'Skyfall', 'The Hobbit: An

In [206]:
cleaned = []
for string in missing:
    clean = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),string)
    cleaned.append(clean)
print(cleaned)

try_again = []

['Star Wars Ep. VII: The Force Awakens', 'Pirates of the Caribbean: At World’s End', 'Justice League', 'Spectre', 'Spider-Man 3', 'No Time to Die', 'The Avengers', 'Pirates of the Caribbean: Dead Man’s Chest', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'Tenet', 'Men in Black 3', 'Robin Hood', 'Black Panther', 'Star Wars Ep. VIII: The Last Jedi', 'Spider-Man 2', 'Alice in Wonderland', 'Fast & Furious Presents: Hobbs & Shaw', 'Tron: Legacy', 'The Mummy', 'The Great Gatsby', 'Disney’s A Christmas Carol', 'Pacific Rim', 'Brave', 'The Chronicles of Narnia: The Lion, the Witch a…', 'Mission: Impossible—Fallout', 'The Jungle Book', 'Inside Out', 'Suicide Squad', 'Coco', 'The Mummy: Tomb of the Dragon Emperor', '47 Ronin', 'The Polar Express', 'Tomorrowland', 'Alice Through the Looking Glass', 'Doctor Strange', 'Beauty and the Beast', 'The Hunger Games: Mockingjay - Part 2', 'The Curious Case of Benjamin Button', "The Sorcerer's Apprentice", 'The Chronicles of Narnia: The Voya

In [211]:
formatted = []
for movie in cleaned:
    _ = movie.replace(': ', '_')
    add = _.replace(' ', '_')
    add = add.replace('-', '_')
    add = add.replace('.', '')
    add = add.replace(',', '')
    add = add.replace('_Ep_', '_Episode_')
    
    formatted.append(add)
    
print(formatted)

['Star_Wars_Episode_VII_The_Force_Awakens', 'Pirates_of_the_Caribbean_At_World’s_End', 'Justice_League', 'Spectre', 'Spider_Man_3', 'No_Time_to_Die', 'The_Avengers', 'Pirates_of_the_Caribbean_Dead_Man’s_Chest', 'Man_of_Steel', 'The_Chronicles_of_Narnia_Prince_Caspian', 'Tenet', 'Men_in_Black_3', 'Robin_Hood', 'Black_Panther', 'Star_Wars_Episode_VIII_The_Last_Jedi', 'Spider_Man_2', 'Alice_in_Wonderland', 'Fast_&_Furious_Presents_Hobbs_&_Shaw', 'Tron_Legacy', 'The_Mummy', 'The_Great_Gatsby', 'Disney’s_A_Christmas_Carol', 'Pacific_Rim', 'Brave', 'The_Chronicles_of_Narnia_The_Lion_the_Witch_a…', 'Mission_Impossible—Fallout', 'The_Jungle_Book', 'Inside_Out', 'Suicide_Squad', 'Coco', 'The_Mummy_Tomb_of_the_Dragon_Emperor', '47_Ronin', 'The_Polar_Express', 'Tomorrowland', 'Alice_Through_the_Looking_Glass', 'Doctor_Strange', 'Beauty_and_the_Beast', 'The_Hunger_Games_Mockingjay___Part_2', 'The_Curious_Case_of_Benjamin_Button', "The_Sorcerer's_Apprentice", 'The_Chronicles_of_Narnia_The_Voyage_of

In [221]:
# Grab year
def get_year(series):
    '''
    Take in date Series and clean to use for link generation on rt.
    Returns list of years in str form
    '''
    temp = []
    for date in series:
        try:
            val = date.replace(' ','').split(',')
            temp.append(val[1])
        except:
            temp.append(val[0])
    return temp

In [223]:
# Pull in Budgets DF
budgets_pd_orig = pd.read_csv('data/budgets_pd.csv')

In [225]:
# years = get_year(budgets_pd_orig['date'])

In [ ]:
# https://www.rottentomatoes.com/m/pirates_of_the_caribbean_3 is at world's end
# https://www.rottentomatoes.com/m/spectre_2015 for spectre
# https://www.rottentomatoes.com/m/justice_league_2017
# https://www.rottentomatoes.com/m/the_great_gatsby_2013